In [1]:
from dotenv import load_dotenv
import os
from supabase import create_client
import fonctions.fonctions_api as f

C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

load_dotenv()  # charge le .env
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
season = os.getenv("SEASON")
supabase = create_client(url, key)

In [ ]:
f.acheter_joueur(supabase, id_user = 3, id_contrat = 335)

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Deadline?select=START%2CEND&START=lte.2025-05-02T19%3A25%3A07.081826%2B02%3A00&END=gte.2025-05-02T19%3A25%3A07.081826%2B02%3A00 "HTTP/2 200 OK"


False

In [3]:
f.ajouter_user(supabase, pseudo = "gpain1999", mot_de_passe = "Iledefrance75@", adresse_mail = "guillaume.pain9@gmail.com")

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/User?select=adresse_mail&adresse_mail=eq.guillaume.pain9%40gmail.com "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/User?select=pseudo&pseudo=eq.gpain1999 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/User "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Banque "HTTP/2 201 Created"


✅ Utilisateur gpain1999 ajouté avec id 4 et solde initial 110


In [14]:
from euroleague_api.game_stats import GameStats

gs = GameStats()

ggs = gs.get_game_report(season=season, game_code=324)

In [3]:
f.get_update_match_data(supabase, season)

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Match?select=id_match&season=eq.2024 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Match?select=id_match&season=eq.2024 "HTTP/2 200 OK"



🔄 Traitement du match 323...


INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier "HTTP/2 201 Created"


✅ Match ajouté au calendrier : BAR vs MCO (Round 40)
⚠️ Match 323 non joué, pas de données disponibles.

🔄 Traitement du match 324...


INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier "HTTP/2 201 Created"


...

✅ Match ajouté au calendrier : PAN vs IST (Round 41)
⚠️ Match 328 non joué, pas de données disponibles.

🔄 Traitement du match 329...

❌ Erreur dans game_code 329 → 404 Client Error: Not Found for url: https://api-live.euroleague.net/v3/competitions/E/seasons/E2024/games/329/report

🔄 Traitement du match 330...

❌ Erreur dans game_code 330 → 404 Client Error: Not Found for url: https://api-live.euroleague.net/v3/competitions/E/seasons/E2024/games/330/report

🔄 Traitement du match 331...

❌ Erreur dans game_code 331 → 404 Client Error: Not Found for url: https://api-live.euroleague.net/v3/competitions/E/seasons/E2024/games/331/report

🔄 Traitement du match 332...

❌ Erreur dans game_code 332 → 404 Client Error: Not Found for url: https://api-live.euroleague.net/v3/competitions/E/seasons/E2024/games/332/report

🔄 Traitement du match 333...


INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Match?select=id_match%2Cseason%2Cround "HTTP/2 200 OK"
INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier?id_match=eq.2&season=eq.2024&round=eq.1 "HTTP/2 200 OK"



❌ Erreur dans game_code 333 → 404 Client Error: Not Found for url: https://api-live.euroleague.net/v3/competitions/E/seasons/E2024/games/333/report


INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier?id_match=eq.4&season=eq.2024&round=eq.1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier?id_match=eq.5&season=eq.2024&round=eq.1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier?id_match=eq.6&season=eq.2024&round=eq.1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier?id_match=eq.7&season=eq.2024&round=eq.1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier?id_match=eq.8&season=eq.2024&round=eq.1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier?id_match=eq.9&season=eq.2024&round=eq.1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Calendrier?id_match=eq.10&seaso

✅ Calendrier nettoyé : doublons supprimés par correspondance avec Match.
